In [1]:
from utils import get_processed_metadata
import pandas as pd
from datetime import datetime
import re
from typing import Iterable, List, Set
from datetime import datetime, timezone, timedelta

In [2]:
processed_data_df = get_processed_metadata("./openai_processed_data")
#processed_data_df

⚠️  JSON parse failed in batch_683c548226b481908de6d9ab7d6fab0c_output.jsonl: Expecting ',' delimiter: line 1 column 4870 (char 4869)
--- Content snippet ---
[{"tweet_id":"@saiful_chowdury_2015-10-24T01:24:57.000Z","row_num":4204849,"category":"Technological","subcategory":"Materials","sentiment":"Positive","tweet_language":"English"},{"tweet_id":"@saiful_...

⚠️  Final fallback skipped object for tweet_id @saiful_chowdury_2015-11-25T20:24:00.000Z in batch_683c548226b481908de6d9ab7d6fab0c_output.jsonl: Expecting ',' delimiter: line 1 column 56 (char 55)
⚠️  Final fallback skipped object for tweet_id @saiful_chowdury_2016-04-22T21:38:27.000Z in batch_683c548226b481908de6d9ab7d6fab0c_output.jsonl: Expecting ',' delimiter: line 1 column 56 (char 55)
⚠️  Final fallback skipped object for tweet_id @saiful_chowdury_2016-03-25T19:18:57.000Z in batch_683c548226b481908de6d9ab7d6fab0c_output.jsonl: Expecting ',' delimiter: line 1 column 56 (char 55)
⚠️  Final fallback skipped object for tweet_id

In [3]:
# 1️⃣  — show duplicates (based on tweet_id)
dupes = processed_data_df[
    processed_data_df.duplicated(subset="tweet_id", keep=False)  
].sort_values("tweet_id") 

print(f"{len(dupes)} duplicated rows found")
dupes.head()    

processed_data_df = (
    processed_data_df
        .drop_duplicates(subset="tweet_id", keep="first")
        .reset_index(drop=True)    # tidy re-index
)

746 duplicated rows found


In [4]:
### Validate That

In [5]:
# ---------------------------------------------------------------------------
# (Optional) master list of allowed sub-categories taken from your prompt
ALLOWED_SUBCATEGORIES: dict[str, List[str]] = {
    "Use-case": [
        "Motor Vehicles / Automotive",
        "Aerospace",
        "Industrial / Business Machines",
        "Consumer Products / Electronics",
        "Medical / Dental",
        "Academic Institutions",
        "Government / Military",
        "Architectural",
        "Power / Energy",
        "Home & DIY (Consumer / Hobbyist)",
        "Other",
    ],
    "Business-relevant": [
        "Supply Chain, Manufacturing & Logistics",
        "Cost Models & Pricing",
        "Intellectual Property & Patents",
        "Mergers, Acquisitions & Partnerships",
        "Investment & Financing",
        "Business Models",
        "Customer Adoption & Demand Dynamics",
        "Sustainability & Circular Economy",
        "Market Trends & Forecast",
        "Other",
    ],
    "Technological": [
        "Materials",
        "Printing Processes",
        "Hardware & Equipment",
        #"Software & Design Tools",
        "Software, Firmware & Design Tools",
        "Hardware, Sensors & Equipment",
        "Process Monitoring & Control",
        "Post-Processing Techniques",
        "AI & Digital Twin Integration",
        "Other",
    ],
    "N/A": ["N/A"],
}

def slug(text: str) -> str:
    """Turn 'Use-case' → 'use_case', 'Technological' → 'technological', etc."""
    return re.sub(r"[^\w]+", "_", text.strip().lower()).strip("_")
    
# ── Add the shorter variant so both spellings are accepted ──────────────
ALLOWED_SUBCATEGORIES["Use-case"].append("Home & DIY")
ALLOWED_SUBCATEGORIES["Use-case"].append("Business Machines")
ALLOWED_SUBCATEGORIES["Technological"].append("Design Tools")
"Processes Monitoring & Control"
# ---------------------------------------------------------------------------


def find_invalid_subcategories(
    df: pd.DataFrame,
    main_category: str,
    valid_subcategories: Iterable[str] | None = None,
    *,
    main_col: str = "category",
    sub_col: str = "subcategory",
) -> pd.DataFrame:
    """
    Return all records where `main_col` == main_category but `sub_col`
    is NOT in the allowed list.

    Parameters
    ----------
    df : pandas.DataFrame
        Your processed tweets.
    main_category : str
        The top-level category to test (e.g. "Use-case").
    valid_subcategories : Iterable[str] | None, default None
        Explicit list of allowed subs.  If None, the function will look up
        `ALLOWED_SUBCATEGORIES[main_category]`.
    main_col : str, default "classification"
        Column holding the top-level category.
    sub_col : str,  default "subclassification"
        Column holding the sub-category.

    Returns
    -------
    pandas.DataFrame
        All violating rows (same columns as the original `df`).
    """
    if valid_subcategories is None:
        try:
            valid_subcategories = ALLOWED_SUBCATEGORIES[main_category]
        except KeyError as e:
            raise ValueError(
                f"No built-in sub-category list for {main_category!r}; "
                "please supply `valid_subcategories` explicitly."
            ) from e

    mask_main = df[main_col] == main_category
    mask_bad = ~df[sub_col].isin(valid_subcategories)
    return df.loc[mask_main & mask_bad]


def save_bad_rows(
    df: pd.DataFrame,
    main_category: str,
    *,
    main_col: str = "category",
    sub_col: str = "subcategory",
) -> str:
    """
    • Runs find_invalid_subcategories() for `main_category`
    • Saves the result as CSV with   bad_<slug>_<YYYY-MM-DD_HHMMSS>.csv
    • Returns the file name created
    """
    bad_rows = find_invalid_subcategories(
        df,
        main_category,
        main_col=main_col,
        sub_col=sub_col,
    )

    timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")
    fname = f"bad_{slug(main_category)}_{timestamp}.csv"
    bad_rows.to_csv(fname, index=False, encoding="utf-8")
    return fname, bad_rows


In [6]:

#bad_file = save_bad_rows(processed_data_df, "Use-case")
fname, inaccurate_rows_business = save_bad_rows(processed_data_df, "Business-relevant")
inaccurate_rows_business

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
119,@KnowNOW_Knowhow_2012-09-02 08:16:47+00:00,94268.0,Business-relevant,Academic Institutions,Neutral,English
139,@3Ddruckmag_ch_2017-03-21T14:56:26.000Z,1188741.0,Business-relevant,Partnerships & Mergers,Positive,English
145,@techupdate14_2016-09-15T13:52:34.000Z,999040.0,Business-relevant,Partnerships & Mergers,Neutral,English
300,@S1lverArrow_2016-11-24T03:48:13.000Z,715820.0,Business-relevant,Use Models & Pricing,Neutral,English
325,@MouldsPlastics_2017-03-29T09:47:13.000Z,1980775.0,Business-relevant,Partnerships & Mergers,Positive,English
...,...,...,...,...,...,...
2823046,@3dnoisedotcom_2015-11-05T06:42:52.000Z,3782792.0,Business-relevant,Crowdfunding & Kickstarter,Neutral,English
2823597,@agwestbio_2014-09-16T21:41:13.000Z,5394030.0,Business-relevant,Academic Institutions,Neutral,English
2824190,@airfoilgroup_2019-11-27T17:14:03.000Z,2657742.0,Business-relevant,Partnerships,Neutral,English
2825201,@AdamJPenna_2016-12-13T16:44:37.000Z,2283933.0,Business-relevant,Partnerships,Positive,English


In [7]:

#bad_file = save_bad_rows(processed_data_df, "Use-case")
fname, inaccurate_rows_tech = save_bad_rows(processed_data_df, "Technological")
inaccurate_rows_tech

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
8,@MakerClub_2014-05-06T14:42:13.000Z,5335696.0,Technological,"Hardware,Sensors & Equipment",Positive,English
15,@RDMolders_2015-05-26T14:25:27.000Z,4062330.0,Technological,"Hardware,Sensors & Equipment",Positive,English
20,@TheDailyEngage_2015-04-19T20:00:36.000Z,4456162.0,Technological,"Hardware,Sensors & Equipment",Positive,English
23,@Solutions_Cubed_2013-07-29T17:01:55.000Z,5707641.0,Technological,"Hardware,Sensors & Equipment",Positive,English
25,@PaulBenjou_2014-04-30T12:45:14.000Z,5331095.0,Technological,"Hardware,Sensors & Equipment",Positive,English
...,...,...,...,...,...,...
2825958,@deltamaker_2019-10-16T14:53:32.000Z,2748811.0,Technological,"Hardware,Sensors & Equipment",Positive,English
2825960,@Afinia3DPrint_2012-07-13 20:36:47+00:00,133493.0,Technological,"Hardware,Sensors & Equipment",Neutral,English
2825967,@3Dprint_Seattle_2016-03-13T13:39:18.000Z,2021109.0,Technological,"Hardware,Sensors & Equipment",Neutral,English
2825980,@Rio_Chic_2014-01-29T09:54:41.000Z,5148358.0,Technological,"Hardware,Sensors & Equipment",Positive,English


In [8]:

#bad_file = save_bad_rows(processed_data_df, "Use-case")
fname, inaccurate_rows_usecase = save_bad_rows(processed_data_df, "Use-case")
inaccurate_rows_usecase

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
733,@s3xyg33k81_2016-01-09T18:29:33.000Z,1161987.0,Use-case,Military / Government,Neutral,English
734,@lpcrepairs_2015-10-18T16:36:34.000Z,4003748.0,Use-case,Military / Government,Neutral,English
1390,@Beckatt_2015-07-25T03:23:06.000Z,3851035.0,Use-case,Sport / Recreation,Positive,English
3330,@areebairfan_nhs_2018-05-23T01:43:02.000Z,3485623.0,Use-case,Use-case,Positive,English
3358,@jacklesnik_2014-10-03T18:46:56.000Z,4873480.0,Use-case,Technological,Neutral,English
...,...,...,...,...,...,...
2823598,@greenstocknews_2016-09-14T18:02:22.000Z,1495831.0,Use-case,Business Models,Neutral,English
2824613,@m_acharia_2013-05-04T18:44:56.000Z,6225597.0,Use-case,Business Models,Positive,English
2824621,@mjneutel_2013-11-08T15:11:46.000Z,5893266.0,Use-case,Military / Government,Neutral,English
2824623,@fineguyg_2013-05-06T17:11:35.000Z,5910190.0,Use-case,Military / Government,Neutral,English


## 1.2 SubCAtegory Standardization and cleanup

In [9]:
## 1.2 SubCAtegory Standardization and cleanup

processed_data_df.loc[
    (processed_data_df["category"] == "Business-relevant") &
    (processed_data_df["subcategory"].isin(["Partnerships & Mergers", "Partnerships"])),
    "subcategory"
] = "Mergers, Acquisitions & Partnerships"


processed_data_df.loc[
    (processed_data_df["category"] == "Business-relevant") &
    (processed_data_df["subcategory"].isin(["Partnerships & Mergers", "Partnerships", 
                                            "Partnerships & Acquisitions", "Partnerships & Mergers & Acquisitions",
                                            "Partnerships (Mergers, Acquisitions & Partnerships)",
                                            "Partnerships & Collaborations", "Partnerships / Mergers & Acquisitions",
                                            "Partnerships And Collaborations", "Partnerships - merged into Mergers, Acquisitions & Partnerships",
                                            "Partnerships, Acquisitions & Partnerships","Partnerships & Alliances", "Partnerships / Mergers"
                                            
                                            "Partnerships, Mergers & Acquisitions"])),
    "subcategory"
] = "Mergers, Acquisitions & Partnerships"


# Normalize all Business-relevant subcategories that mention "partnership" (in any form or case)
# by replacing them with the unified label "Mergers, Acquisitions & Partnerships"
processed_data_df.loc[
    (processed_data_df["category"] == "Business-relevant") &
    (processed_data_df["subcategory"].str.contains(r"\bpartnership", case=False, na=False)),
    "subcategory"
] = "Mergers, Acquisitions & Partnerships"


# Normalize all Business-relevant subcategories that mention "Market Trends" 
# by replacing them with the unified label "Market Trends & Forecast"
processed_data_df.loc[
    (processed_data_df["category"] == "Business-relevant") &
    (processed_data_df["subcategory"].str.contains(r"Market Trends", case=False, na=False)),
    "subcategory"
] = "Market Trends & Forecast"


# Normalize all Business-relevant subcategories that mention "business model", "business models", or "business modes"
# by replacing them with the unified label "Business Models"
processed_data_df.loc[
    (processed_data_df["category"] == "Business-relevant") &
    (processed_data_df["subcategory"].str.contains(r"business model[s]?|business modes?", case=False, na=False)),
    "subcategory"
] = "Business Models"


### Technological Standardization

# Normalize all Technological subcategories that mention "Firmware & Design Tools"
# by replacing them with the unified label "Software, Firmware & Design Tools"
processed_data_df.loc[
    (processed_data_df["category"] == "Technological") &
    (processed_data_df["subcategory"].str.contains(r"Firmware & Design Tools", case=False, na=False)),
    "subcategory"
] = "Software, Firmware & Design Tools"

##---
# Normalize all Technological subcategories that mention either "Firmware & Design Tools" or "Software & Design Tools"
# by replacing them with the unified label "Software, Firmware & Design Tools"
processed_data_df.loc[
    (processed_data_df["category"] == "Technological") &
    (processed_data_df["subcategory"].str.contains(r"(Firmware|Software) & Design Tools", case=False, na=False)),
    "subcategory"
] = "Software, Firmware & Design Tools"

###---
# Normalize  all Technological subcategories that mention "Sensors & Equipment"
# by replacing them with the unified label "Hardware, Sensors & Equipment"
processed_data_df.loc[
    (processed_data_df["category"] == "Technological") &
    (processed_data_df["subcategory"].str.contains(r"Sensors & Equipment", case=False, na=False)),
    "subcategory"
] = "Hardware, Sensors & Equipment"


###---
# Normalize all Technological subcategories that contain "material" or "materials" (case-insensitive)
# by replacing them with the unified label "Materials"
processed_data_df.loc[
    (processed_data_df["category"] == "Technological") &
    (processed_data_df["subcategory"].str.contains(r"\bmaterials?\b", case=False, na=False)),
    "subcategory"
] = "Materials"


###---
# Normalize all Technological subcategories that contain "Firmware" (case-insensitive)
# by replacing them with the unified label "Software, Firmware & Design Tools"
processed_data_df.loc[
    (processed_data_df["category"] == "Technological") &
    (processed_data_df["subcategory"].str.contains(r"\bfirmware\b", case=False, na=False)),
    "subcategory"
] = "Software, Firmware & Design Tools"


###---
# Normalize all Technological subcategories that contain "Sensor" or "Sensors" (case-insensitive)
# by replacing them with the unified label "Hardware, Sensors & Equipment"
processed_data_df.loc[
    (processed_data_df["category"] == "Technological") &
    (processed_data_df["subcategory"].str.contains(r"\bsensors?\b", case=False, na=False)),
    "subcategory"
] = "Hardware, Sensors & Equipment"


###---
# Normalize all Technological subcategories that contain "Firmware & Software" (case-insensitive)
# by replacing them with the unified label "Software, Firmware & Design Tools"
processed_data_df.loc[
    (processed_data_df["category"] == "Technological") &
    (processed_data_df["subcategory"].str.contains(r"Firmware & Software", case=False, na=False)),
    "subcategory"
] = "Software, Firmware & Design Tools"


###---
# Normalize all Technological subcategories that contain the word "Firmware" (case-insensitive)
# by replacing them with the unified label "Software, Firmware & Design Tools"
processed_data_df.loc[
    (processed_data_df["category"] == "Technological") &
    (processed_data_df["subcategory"].str.contains(r"\bfirmware\b", case=False, na=False)),
    "subcategory"
] = "Software, Firmware & Design Tools"



/tmp/ipykernel_256871/3674317947.py:66: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (processed_data_df["subcategory"].str.contains(r"(Firmware|Software) & Design Tools", case=False, na=False)),


In [10]:
# Normalize all Use-case subcategories that contain "Military" or "Government" (case-insensitive)
# by replacing them with the unified label "Government / Military"
processed_data_df.loc[
    (processed_data_df["category"] == "Use-case") &
    (processed_data_df["subcategory"].str.contains(r"\b(military|government)\b", case=False, na=False)),
    "subcategory"
] = "Government / Military"


/tmp/ipykernel_256871/2057433124.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  (processed_data_df["subcategory"].str.contains(r"\b(military|government)\b", case=False, na=False)),


In [11]:
# Fix formatting: update subcategory "Hardware,Sensors & Equipment" to "Hardware, Sensors & Equipment"
processed_data_df.loc[
    processed_data_df["subcategory"] == "Hardware,Sensors & Equipment",
    "subcategory"
] = "Hardware, Sensors & Equipment"


In [12]:

#bad_file = save_bad_rows(processed_data_df, "Use-case")
fname, inaccurate_rows_business = save_bad_rows(processed_data_df, "Business-relevant")
inaccurate_rows_business

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
119,@KnowNOW_Knowhow_2012-09-02 08:16:47+00:00,94268.0,Business-relevant,Academic Institutions,Neutral,English
300,@S1lverArrow_2016-11-24T03:48:13.000Z,715820.0,Business-relevant,Use Models & Pricing,Neutral,English
354,@3dtemplates_2021-04-12T22:40:33.000Z,2838093.0,Business-relevant,Industrial / Business Machines,Neutral,English
362,@dynamism_2023-09-26T17:00:57.000Z,6352106.0,Business-relevant,Academic Institutions,Neutral,English
535,@GHSEngineering_2014-02-28T17:58:00.000Z,4705826.0,Business-relevant,Academic Institutions,Positive,English
...,...,...,...,...,...,...
2821925,@bcn3dtech_2022-11-26T12:36:57.000Z,6470757.0,Business-relevant,Innovation,Positive,English
2822570,@TunedPR_2014-08-12T20:37:18.000Z,5530809.0,Business-relevant,Marketing & Branding,Neutral,English
2822628,@sanilsundar_2014-10-29T17:38:49.000Z,4821170.0,Business-relevant,Competitive Dynamics & Customer Adoption,Positive,English
2823046,@3dnoisedotcom_2015-11-05T06:42:52.000Z,3782792.0,Business-relevant,Crowdfunding & Kickstarter,Neutral,English


In [13]:

#bad_file = save_bad_rows(processed_data_df, "Use-case")
fname, inaccurate_rows_technological = save_bad_rows(processed_data_df, "Technological")
inaccurate_rows_technological

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
80,@mUVe3DLLC_2016-12-08T16:34:31.000Z,1992236.0,Technological,Intellectual Property & Patents,Positive,English
81,@BigNerdMedia_2016-12-09T16:55:09.000Z,778900.0,Technological,Intellectual Property & Patents,Positive,English
148,@dailycadcam_2016-12-09T05:45:24.000Z,1549538.0,Technological,Intellectual Property & Patents,Positive,English
149,@divbyztech_2016-12-08T13:59:30.000Z,1992444.0,Technological,Intellectual Property & Patents,Positive,English
521,@Inventables_2014-01-27T17:21:00.000Z,4996383.0,Technological,Use-case,Positive,English
...,...,...,...,...,...,...
2817081,@Tampa_Musicman_2015-07-20T13:24:27.000Z,4283132.0,Technological,Consumer Products / Electronics,Positive,English
2817200,@celesthar_2015-05-03T03:28:15.000Z,4516952.0,Technological,Cost Models & Pricing,Neutral,English
2823746,@macohibs_2013-12-18T08:08:03.000Z,5984214.0,Technological,Use-case,Positive,English
2825010,@Alltop_Opera_2013-01-09T23:40:16.000Z,5847808.0,Technological,Use-case,Neutral,English


In [14]:

#bad_file = save_bad_rows(processed_data_df, "Use-case")
fname, inaccurate_rows_usecase = save_bad_rows(processed_data_df, "Use-case")
inaccurate_rows_usecase

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
1390,@Beckatt_2015-07-25T03:23:06.000Z,3851035.0,Use-case,Sport / Recreation,Positive,English
3330,@areebairfan_nhs_2018-05-23T01:43:02.000Z,3485623.0,Use-case,Use-case,Positive,English
3358,@jacklesnik_2014-10-03T18:46:56.000Z,4873480.0,Use-case,Technological,Neutral,English
3368,@MakoInvent_2020-02-19T15:00:41.000Z,519215.0,Use-case,Use-case,Positive,English
3699,@bukiziwybex_2014-08-01T12:17:47.000Z,4852035.0,Use-case,Use-case,Neutral,English
...,...,...,...,...,...,...
2822374,@mMargretteRose_2013-11-21T22:11:20.000Z,5704545.0,Use-case,Business Models,Negative,English
2822955,@MrCodiga_2017-12-28T19:25:54.000Z,918244.0,Use-case,Mechanical / Industrial Machines,Neutral,English
2823437,@carlix123_2015-12-24T19:59:47.000Z,3756691.0,Use-case,Sustainability & Circular Economy,Positive,English
2823598,@greenstocknews_2016-09-14T18:02:22.000Z,1495831.0,Use-case,Business Models,Neutral,English


## 2. Find Invalid Categories

In [15]:
def find_invalid_categories(
    df: pd.DataFrame,
    *,
    main_col: str = "category",
    allowed: Iterable[str] | None = None,
) -> pd.DataFrame:
    """
    Return every row whose top-level `category` is NOT one of the 3 valid values.

    Parameters
    ----------
    df : pandas.DataFrame
    main_col : str            column holding the top-level category (default: "category")
    allowed : iterable[str]   list or set of valid categories; if None, defaults to
                              {"Use-case", "Business-relevant", "Technological"}

    Returns
    -------
    pandas.DataFrame          rows with an out-of-scope top-level category
    """
    if allowed is None:
        allowed: Set[str] = {"Use-case", "Business-relevant", "Technological", "N/A"}

    return df.loc[~df[main_col].isin(allowed)].copy()


In [16]:
bad_cat_rows = find_invalid_categories(processed_data_df)

bad_cat_rows

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
581,@Wamungo3DPrint_2014-01-19T00:34:44.000Z,4869497.0,"Software, Firmware & Design Tools","Software, Firmware & Design Tools",Neutral,English
610,@tollsy_2014-03-29T16:51:04.000Z,4667710.0,Academic Institutions,Use-case,Neutral,English
3163,@animationcollab_2015-11-16T22:27:09.000Z,3930296.0,Academic Institutions,Academic Institutions,Positive,English
3516,@asesftw_tech_2019-12-16T02:30:24.000Z,2329190.0,Academic Institutions,Academic Institutions,Positive,English
3530,@BucknellLIT_2014-03-21T19:04:30.000Z,5618247.0,Academic Institutions,Academic Institutions,Neutral,English
...,...,...,...,...,...,...
2822722,@GrangeInd_2018-01-27T11:45:05.000Z,3618050.0,Academic Institutions,Other,Positive,English
2823662,@Jilleeanann_2013-10-21T21:09:28.000Z,5772725.0,Government / Military,Government / Military,Neutral,English
2824556,@WitsADDP_2014-07-04T17:28:32.000Z,5541977.0,Academic Institutions,Other,Neutral,English
2825527,@JHornick3D1Stop_2020-07-07T15:30:27.000Z,587234.0,Other,N/A,Neutral,English


## 2.2 Categories Standardization

In [17]:
# If subcategory is "Academic Institutions" or "Use-case", and category is "Academic Institutions",
# update the category to "Use-case"
processed_data_df.loc[
    (processed_data_df["subcategory"].isin(["Academic Institutions", "Use-case"])) &
    (processed_data_df["category"] == "Academic Institutions"),
    "category"
] = "Use-case"


In [18]:
# If subcategory is "Government / Military" or "Use-case", and category is "Government / Military",
# update the category to "Use-case"
processed_data_df.loc[
    (processed_data_df["subcategory"].isin(["Government / Military", "Use-case"])) &
    (processed_data_df["category"] == "Government / Military"),
    "category"
] = "Use-case"


In [19]:
# Replace all category values that are "Other" or "Others" with "N/A"
processed_data_df.loc[
    processed_data_df["category"].isin(["Other", "Others"]),
    "category"
] = "N/A"


In [20]:
# If category is "Academic Institutions", update category to "Use-case" and subcategory to "Academic Institutions"
processed_data_df.loc[
    processed_data_df["category"] == "Academic Institutions",
    ["category", "subcategory"]
] = ["Use-case", "Academic Institutions"]


In [21]:
# If category is "Software, Firmware & Design Tools", update category to "Technological"
# and subcategory to "Software, Firmware & Design Tools"
processed_data_df.loc[
    processed_data_df["category"] == "Software, Firmware & Design Tools",
    ["category", "subcategory"]
] = ["Technological", "Software, Firmware & Design Tools"]


In [22]:
# If category is "Government / Military", update category to "Use-case"
# and subcategory to "Government / Military"
processed_data_df.loc[
    processed_data_df["category"] == "Government / Military",
    ["category", "subcategory"]
] = ["Use-case", "Government / Military"]


In [23]:
# If category contains "Intellectual Property" (case-insensitive),
# update category to "Business-relevant" and subcategory to "Intellectual Property & Patents"
processed_data_df.loc[
    processed_data_df["category"].str.contains(r"Intellectual Property", case=False, na=False),
    ["category", "subcategory"]
] = ["Business-relevant", "Intellectual Property & Patents"]


In [24]:
# If category contains "Hardware", "Sensor", or "Equipment" (case-insensitive),
# update category to "Technological" and subcategory to "Hardware, Sensors & Equipment"
processed_data_df.loc[
    processed_data_df["category"].str.contains(r"hardware|sensor|equipment", case=False, na=False),
    ["category", "subcategory"]
] = ["Technological", "Hardware, Sensors & Equipment"]


In [25]:
# If category contains "Mergers", "Acquisition", or "Partnerships" (case-insensitive),
# update category to "Business-relevant" and subcategory to "Mergers, Acquisitions & Partnerships"
processed_data_df.loc[
    processed_data_df["category"].str.contains(r"mergers|acquisition|partnerships", case=False, na=False),
    ["category", "subcategory"]
] = ["Business-relevant", "Mergers, Acquisitions & Partnerships"]


In [26]:
# If category is "Architectural", update category to "Use-case" and subcategory to "Architectural"
processed_data_df.loc[
    processed_data_df["category"] == "Architectural",
    ["category", "subcategory"]
] = ["Use-case", "Architectural"]


In [27]:
# If category is "Educational", update category to "Use-case" and subcategory to "Academic Institutions"
processed_data_df.loc[
    processed_data_df["category"] == "Educational",
    ["category", "subcategory"]
] = ["Use-case", "Academic Institutions"]


In [28]:
# If category is "Consumer Products / Electronics", update category to "Use-case" and subcategory to "Consumer Products / Electronics"
processed_data_df.loc[
    processed_data_df["category"] == "Consumer Products / Electronics",
    ["category", "subcategory"]
] = ["Use-case", "Consumer Products / Electronics"]


In [29]:
# If category is "Medical / Dental", update category to "Use-case" and subcategory to "Medical / Dental"
processed_data_df.loc[
    processed_data_df["category"] == "Medical / Dental",
    ["category", "subcategory"]
] = ["Use-case", "Medical / Dental"]


In [30]:
# If category is "Home & DIY (Consumer / Hobbyist)", update category to "Use-case" and subcategory to "Home & DIY (Consumer / Hobbyist)"
processed_data_df.loc[
    processed_data_df["category"] == "Home & DIY (Consumer / Hobbyist)",
    ["category", "subcategory"]
] = ["Use-case", "Home & DIY (Consumer / Hobbyist)"]


In [31]:
# Replace all category values that contain "Technology" (case-insensitive) with "Technological"
processed_data_df.loc[
    processed_data_df["category"].str.contains(r"technology", case=False, na=False),
    "category"
] = "Technological"


In [32]:
# If category is "Investment & Financing", update category to "Business-relevant"
# and subcategory to "Investment & Financing"
processed_data_df.loc[
    processed_data_df["category"] == "Investment & Financing",
    ["category", "subcategory"]
] = ["Business-relevant", "Investment & Financing"]


In [33]:
# If category is "Market Trends & Forecast", update category to "Business-relevant"
# and subcategory to "Market Trends & Forecast"
processed_data_df.loc[
    processed_data_df["category"] == "Market Trends & Forecast",
    ["category", "subcategory"]
] = ["Business-relevant", "Market Trends & Forecast"]


In [34]:
# If category is "Materials", update category to "Technological" and subcategory to "Materials"
processed_data_df.loc[
    processed_data_df["category"] == "Materials",
    ["category", "subcategory"]
] = ["Technological", "Materials"]


In [35]:
# If category contains "Educational" (case-insensitive), update category to "Use-case"
# and subcategory to "Academic Institutions"
processed_data_df.loc[
    processed_data_df["category"].str.contains(r"educational", case=False, na=False),
    ["category", "subcategory"]
] = ["Use-case", "Academic Institutions"]


In [36]:
# If category is "Aerospace", update category to "Use-case" and subcategory to "Aerospace"
processed_data_df.loc[
    processed_data_df["category"] == "Aerospace",
    ["category", "subcategory"]
] = ["Use-case", "Aerospace"]


In [37]:
# If category is "Customer Adoption & Demand Dynamics", update category to "Business-relevant"
# and subcategory to "Customer Adoption & Demand Dynamics"
processed_data_df.loc[
    processed_data_df["category"] == "Customer Adoption & Demand Dynamics",
    ["category", "subcategory"]
] = ["Business-relevant", "Customer Adoption & Demand Dynamics"]


In [38]:
# If category is "Motor Vehicles / Automotive", update category to "Use-case"
# and subcategory to "Motor Vehicles / Automotive"
processed_data_df.loc[
    processed_data_df["category"] == "Motor Vehicles / Automotive",
    ["category", "subcategory"]
] = ["Use-case", "Motor Vehicles / Automotive"]


In [39]:
bad_cat_rows = find_invalid_categories(processed_data_df)

bad_cat_rows

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
7691,@Mcknight_6_2015-01-02T22:47:18.000Z,3749448.0,Neutral,N/A,Neutral,English
31886,@alan_rowse_2018-08-31T14:21:59.000Z,3170393.0,Positive,N/A,Positive,English
40924,@emonpaul_2016-03-26T04:33:24.000Z,722505.0,Neutral,N/A,Positive,English
100254,@Wilder_artists_2013-10-12T16:58:06.000Z,5757399.0,Positive,N/A,Positive,English
102330,@supermineadam_2014-10-04T06:40:11.000Z,5527728.0,Positive,N/A,Positive,English
...,...,...,...,...,...,...
2686642,@petro_sino_2020-11-26T12:44:40.000Z,543469.0,Positive,N/A,Positive,English
2713737,@beeekey_2013-09-19T20:33:48.000Z,6197789.0,Positive,N/A,Positive,English
2721751,@HighEnd_France1_2013-11-16T10:59:12.000Z,6107862.0,Neutral,N/A,Neutral,English
2790890,@MeerDerrick_2016-08-02T10:45:09.000Z,795478.0,Positive,Other,N/A,English


In [40]:
# If subcategory is "Business Models", update category to "Business-relevant"
processed_data_df.loc[
    processed_data_df["subcategory"] == "Business Models",
    "category"
] = "Business-relevant"
# Normalize all academic-related category labels to "Use-case" and subcategory to "Academical Institutions"
processed_data_df.loc[
    processed_data_df["category"].isin([
        "Academical Institutions",
        "Education",
        "Education/Academic",
        "Research / Academic Institutions",
        "Academics"
    ]),
    ["category", "subcategory"]
] = ["Use-case", "Academical Institutions"]


In [41]:
# If category is "Sustainability & Circular Economy", update category to "Business-relevant"
# and subcategory to "Sustainability & Circular Economy"
processed_data_df.loc[
    processed_data_df["category"] == "Sustainability & Circular Economy",
    ["category", "subcategory"]
] = ["Business-relevant", "Sustainability & Circular Economy"]


In [42]:
# If category is "Cost Models & Pricing", update category to "Business-relevant"
# and subcategory to "Cost Models & Pricing"
processed_data_df.loc[
    processed_data_df["category"] == "Cost Models & Pricing",
    ["category", "subcategory"]
] = ["Business-relevant", "Cost Models & Pricing"]


In [43]:
# If category is "Supply Chain, Manufacturing & Logistics", update category to "Business-relevant"
# and subcategory to "Supply Chain, Manufacturing & Logistics"
processed_data_df.loc[
    processed_data_df["category"] == "Supply Chain, Manufacturing & Logistics",
    ["category", "subcategory"]
] = ["Business-relevant", "Supply Chain, Manufacturing & Logistics"]
# If category contains "Process Monitoring" (case-insensitive), update category to "Technological"
# and subcategory to "Process Monitoring & Control"
processed_data_df.loc[
    processed_data_df["category"].str.contains(r"process monitoring", case=False, na=False),
    ["category", "subcategory"]
] = ["Technological", "Process Monitoring & Control"]


In [44]:
# Drop rows where category is "Positive"
processed_data_df = processed_data_df[processed_data_df["category"] != "Positive"]
# Drop rows where category is "Neutral"
processed_data_df = processed_data_df[processed_data_df["category"] != "Neutral"]
# Drop rows where category is "Negative"
processed_data_df = processed_data_df[processed_data_df["category"] != "Negative"]



In [45]:
# Normalize variations of "Use-case" in category column
processed_data_df.loc[
    processed_data_df["category"].str.lower() == "used-case",
    "category"
] = "Use-case"

processed_data_df.loc[
    processed_data_df["category"].str.upper() == "USE-CASE",
    "category"
] = "Use-case"
# Correct spelling: "Techological" to "Technological"
# Correct any misspellings or variations to "Technological"
processed_data_df.loc[
    processed_data_df["category"].isin(["Technical", "Techical", "Techological", "Tech"]),
    "category"
] = "Technological"

# If category is "Power / Energy", update category to "Use-case" and subcategory to "Power / Energy"
processed_data_df.loc[
    processed_data_df["category"] == "Power / Energy",
    ["category", "subcategory"]
] = ["Use-case", "Power / Energy"]


In [46]:
# If category is "Post-Processing Techniques" (with or without trailing whitespace),
# update category to "Technological" and subcategory to "Post-Processing Techniques"
processed_data_df.loc[
    processed_data_df["category"].str.strip() == "Post-Processing Techniques",
    ["category", "subcategory"]
] = ["Technological", "Post-Processing Techniques"]


In [47]:
# # If subcategory is "Business Models", update category to "Business-relevant"
# processed_data_df.loc[
#     processed_data_df["subcategory"] == "Business Models",
#     "category"
# ] = "Business-relevant"


In [48]:
# Instead of dropping, set both category and subcategory to "N/A" for unwanted categories
unwanted = [
    "Award / Recognition",
    "Event",
    "Event Announcement",
    "Event announcements",
    "Community / Other",
    "Undetermined",
    "Merging categories (Business-relevant & N/A)",
    "Retail", "Legal and Regulatory", "Non-3D-printing", "History / N/A"
]

processed_data_df.loc[
    processed_data_df["category"].isin(unwanted),
    ["category", "subcategory"]
] = ["N/A", "N/A"]


In [49]:
# try:
#     timestamp
# except NameError:
#     from datetime import datetime
#     timestamp = datetime.now().strftime("%Y-%m-%d_%H%M%S")

# fname = f"bad_top_level_{timestamp}.csv"
# bad_cat_rows.to_csv(fname, index=False, encoding="utf-8")

In [50]:
processed_data_df

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
0,@3HTi__2017-04-13T21:13:00.000Z,1938218.0,Business-relevant,Intellectual Property & Patents,Neutral,English
1,@benzuria_2017-06-12T16:33:14.000Z,1614375.0,Business-relevant,Intellectual Property & Patents,Neutral,English
2,@stephcurcio_2016-02-22T16:27:33.000Z,904891.0,Business-relevant,Intellectual Property & Patents,Positive,English
3,@patesalo_e_2017-06-10T10:26:13.000Z,697479.0,Business-relevant,Intellectual Property & Patents,Neutral,English
4,@eTeknix_2015-04-19T18:04:07.000Z,4456221.0,Use-case,Consumer Products / Electronics,Positive,English
...,...,...,...,...,...,...
2825997,@dgzitting_2016-07-20T22:36:24.000Z,2176450.0,Use-case,Architectural,Positive,English
2825998,@MartaKrivosheek_2015-10-31T10:45:52.000Z,3816338.0,Technological,Materials,Neutral,English
2825999,@anciana_abuela_2018-04-14T16:42:12.000Z,3209381.0,Technological,Materials,Positive,English
2826000,@crimsonns_2022-09-06T16:39:17.000Z,6467551.0,Business-relevant,Investment & Financing,Neutral,English


In [51]:
# from datetime import datetime

# # Generate timestamped filename
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# filename = f"processed_data_df_pre_cleaned_{timestamp}.csv"

# # Save to CSV
# processed_data_df.to_csv(filename, index=False)

# print(f"DataFrame saved as {filename}")


In [52]:
bad_cat_rows = find_invalid_categories(processed_data_df)

bad_cat_rows

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
433267,@mmtsharjah_2022-11-02T08:03:39.000Z,6565900.0,Business Models,Business-relevant,Neutral,Arabic
694468,@AndroidPrintoid_2018-10-02T16:25:38.000Z,3353305.0,Software & Design Tools,Software & Design Tools,Neutral,French
1682355,@Am_notatwit_2014-05-16T20:06:33.000Z,4780643.0,Hobbyist / Consumer,N/A,Neutral,English
2325299,@3Dable_2018-07-30T23:47:12.000Z,3250061.0,Legal/Business-relevant,Intellectual Property & Patents,Negative,English
2506432,@ShockMansion_2013-03-25T23:19:01.000Z,6177886.0,Policy / Government,Government / Military,Neutral,English


In [53]:
# Convert all remaining invalid-category rows to "N/A" for both category and subcategory
processed_data_df.loc[bad_cat_rows.index, ["category", "subcategory"]] = ["N/A", "N/A"]


In [54]:
bad_cat_rows = find_invalid_categories(processed_data_df)

bad_cat_rows

,tweet_id,row_num,category,subcategory,sentiment,tweet_language


## 2.4 Combine the Misclassified Tweets, Remove Them from Processed Tweets and Write them out for reprocessing

In [55]:
# Stack them into a single DataFrame (preserve original indices for removal)
bad_rows_combined = pd.concat(
    [inaccurate_rows_business, inaccurate_rows_technological, inaccurate_rows_usecase]
)


In [56]:
processed_data_df

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
0,@3HTi__2017-04-13T21:13:00.000Z,1938218.0,Business-relevant,Intellectual Property & Patents,Neutral,English
1,@benzuria_2017-06-12T16:33:14.000Z,1614375.0,Business-relevant,Intellectual Property & Patents,Neutral,English
2,@stephcurcio_2016-02-22T16:27:33.000Z,904891.0,Business-relevant,Intellectual Property & Patents,Positive,English
3,@patesalo_e_2017-06-10T10:26:13.000Z,697479.0,Business-relevant,Intellectual Property & Patents,Neutral,English
4,@eTeknix_2015-04-19T18:04:07.000Z,4456221.0,Use-case,Consumer Products / Electronics,Positive,English
...,...,...,...,...,...,...
2825997,@dgzitting_2016-07-20T22:36:24.000Z,2176450.0,Use-case,Architectural,Positive,English
2825998,@MartaKrivosheek_2015-10-31T10:45:52.000Z,3816338.0,Technological,Materials,Neutral,English
2825999,@anciana_abuela_2018-04-14T16:42:12.000Z,3209381.0,Technological,Materials,Positive,English
2826000,@crimsonns_2022-09-06T16:39:17.000Z,6467551.0,Business-relevant,Investment & Financing,Neutral,English


In [57]:
processed_data_df = processed_data_df.drop(index=bad_rows_combined.index)

In [58]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"BAD_ROWS_STACKED_{timestamp}.csv"
bad_rows_combined.to_csv(filename, index=False)

In [60]:
processed_data_df

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
0,@3HTi__2017-04-13T21:13:00.000Z,1938218.0,Business-relevant,Intellectual Property & Patents,Neutral,English
1,@benzuria_2017-06-12T16:33:14.000Z,1614375.0,Business-relevant,Intellectual Property & Patents,Neutral,English
2,@stephcurcio_2016-02-22T16:27:33.000Z,904891.0,Business-relevant,Intellectual Property & Patents,Positive,English
3,@patesalo_e_2017-06-10T10:26:13.000Z,697479.0,Business-relevant,Intellectual Property & Patents,Neutral,English
4,@eTeknix_2015-04-19T18:04:07.000Z,4456221.0,Use-case,Consumer Products / Electronics,Positive,English
...,...,...,...,...,...,...
2825997,@dgzitting_2016-07-20T22:36:24.000Z,2176450.0,Use-case,Architectural,Positive,English
2825998,@MartaKrivosheek_2015-10-31T10:45:52.000Z,3816338.0,Technological,Materials,Neutral,English
2825999,@anciana_abuela_2018-04-14T16:42:12.000Z,3209381.0,Technological,Materials,Positive,English
2826000,@crimsonns_2022-09-06T16:39:17.000Z,6467551.0,Business-relevant,Investment & Financing,Neutral,English


## 3. Merge with Input Data Set and Valiate Tweet ID as well as Row Number

In [61]:
import pandas as pd

def find_rownum_mismatches_df(
    processed_df: pd.DataFrame,
    cleaned_df: pd.DataFrame,
    *,
    tweet_id_col: str = "tweet_id",
    processed_row_col: str = "row_num",
    cleaned_row_col: str = "row_num",
) -> pd.DataFrame:
    """
    Compare row numbers between two in-memory DataFrames that share a tweet_id.

    Parameters
    ----------
    processed_df : pandas.DataFrame
        Output of `get_processed_metadata()` (or equivalent).
    cleaned_df   : pandas.DataFrame
        The original ‘clean’ dataset already loaded into memory.
    tweet_id_col : str
        Column name holding the tweet ID (must exist in both frames).
    processed_row_col : str
        Column holding row numbers in `processed_df`.
    cleaned_row_col   : str
        Column holding row numbers in `cleaned_df`.

    Returns
    -------
    pandas.DataFrame
        Rows where tweet_id matches but row_num differs.
        Columns included:
            tweet_id | row_num_processed | row_num_cleaned | …other processed cols…
    """
    # keep only the columns we need from cleaned_df
    clean_subset = cleaned_df[[tweet_id_col, cleaned_row_col]].copy()
    clean_subset = clean_subset.rename(columns={cleaned_row_col: "row_num_cleaned"})

    proc_subset = processed_df.copy()
    proc_subset = proc_subset.rename(
        columns={processed_row_col: "row_num_processed"}
    )

    merged = proc_subset.merge(
        clean_subset,
        on=tweet_id_col,
        how="inner",
        validate="one_to_one",
    )

    mismatches = merged[
        merged["row_num_processed"] != merged["row_num_cleaned"]
    ].copy()

    return mismatches


In [62]:

cleaned_df = pd.read_csv("CLEANED_DATASET_20250531_004116.csv")

In [63]:
dupe_rows = cleaned_df[cleaned_df.duplicated("tweet_id", keep=False)].copy()
dupe_rows

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text,langdetect_is_english
1818703,@Bigmacncheese_,0,2022-04-28 23:39:12+00:00,@Bigmacncheese__2022-04-28T23:39:12.000Z,6587462,3d printing this would be so sick,True
1818749,@Bigmacncheese_,0,2022-04-28 23:39:12+00:00,@Bigmacncheese__2022-04-28T23:39:12.000Z,6508438,3d printing this would be so sick /jr_bartok/...,True


In [64]:
cleaned_df = cleaned_df.drop_duplicates(subset="tweet_id", keep="first").reset_index(drop=True)

In [65]:
rownum_mismatches = find_rownum_mismatches_df(processed_data_df, cleaned_df)
rownum_mismatches

,tweet_id,row_num_processed,category,subcategory,sentiment,tweet_language,row_num_cleaned
153674,@MakerWatcher_2016-10-27T04:28:04.000Z,1552039.0,Technological,"Software, Firmware & Design Tools",Neutral,English,2171353
162189,@AgedBeyondDecay_2016-12-12T22:02:34.000Z,4600520.0,Use-case,Other,Positive,English,1825285
173131,@Charles_M_Russo_2019-10-07T12:24:33.000Z,2432323.0,Business-relevant,Intellectual Property & Patents,Negative,English,2511089
186854,@fabbing_2015-02-12T10:14:05.000Z,6638174.0,Use-case,Aerospace,Positive,English,4309611
207984,@SpitzStrategy_2015-01-19T16:20:34.000Z,5369850.0,Technological,Materials,Neutral,English,3888378
...,...,...,...,...,...,...,...
2131120,@ExtruderJoe_2017-03-06T21:59:13.000Z,1427442.0,Use-case,Medical / Dental,Positive,English,773229
2139348,@sherrydeclan1_2015-09-28T09:26:53.000Z,4275049.0,N/A,N/A,Neutral,English,4237983
2236935,@msmirza_2016-12-17T10:23:47.000Z,5425015.0,Use-case,Consumer Products / Electronics,Positive,English,1911104
2756420,@scoo_ps_2013-09-10T08:12:30.000Z,5927610.0,Business-relevant,Investment & Financing,Positive,English,6101905


### Drop Mismatched Rows Numbers

In [ ]:
mismatched_ids = rownum_mismatches["tweet_id"].unique()

# Drop every row in processed_data_df whose tweet_id appears in that list
processed_data_df = processed_data_df[
    ~processed_data_df["tweet_id"].isin(mismatched_ids)
]

In [72]:
rownum_mismatches = find_rownum_mismatches_df(processed_data_df, cleaned_df)
rownum_mismatches

,tweet_id,row_num_processed,category,subcategory,sentiment,tweet_language,row_num_cleaned


In [ ]:
# Filter the processed DataFrame for the same tweet_id
maker_row_processed = processed_data_df.loc[
    processed_data_df["tweet_id"] == "@MakerWatcher_2016-10-27T04:28:04.000Z"
]

maker_row_processed

,tweet_id,row_num,category,subcategory,sentiment,tweet_language


## Check For Hallucinated Tweet IDs

In [74]:
def clean_tweet_ids(df: pd.DataFrame, tweet_id_col: str) -> pd.Series:
    return (
        df[tweet_id_col]
        .astype(str)
        .str.strip()
        .str.replace(r'[",]+$', '', regex=True)  # removes trailing , and "
    )
def check_hallucinated_rows(
    processed_df: pd.DataFrame,
    cleaned_df: pd.DataFrame,
    *,
    tweet_id_col: str = "tweet_id"
) -> pd.DataFrame:
    """
    Identify all rows in `processed_df` that are not present in `cleaned_df`
    based on the cleaned `tweet_id` (stripped, no trailing punctuation).

    Returns rows in processed_df where cleaned tweet_id is not in cleaned_df.
    """
    processed_clean = processed_df.copy()
    cleaned_clean = cleaned_df.copy()

    # Clean tweet_id columns
    processed_clean[tweet_id_col] = clean_tweet_ids(processed_clean, tweet_id_col)
    cleaned_clean[tweet_id_col] = clean_tweet_ids(cleaned_clean, tweet_id_col)

    # Find tweet_ids in processed but not in cleaned
    extra_rows = processed_clean[
        ~processed_clean[tweet_id_col].isin(cleaned_clean[tweet_id_col])
    ].copy()

    return extra_rows


In [75]:
extra_rows = check_hallucinated_rows(
    processed_data_df, 
    cleaned_df, 
    tweet_id_col="tweet_id"
)
extra_rows

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
28487,@DscheyH_2015-03-29T23:27:08.000Z,4289779.0,Technological,"Hardware, Sensors & Equipment",Positive,English
33459,@3Dable_2019-10-05T16:51:32.000Z,2906094.0,Technological,"Hardware, Sensors & Equipment",Positive,English
38866,@ronniejacods417_2016-11-14T16:46:01.000Z,947174.0,Use-case,Medical / Dental,Positive,English
40550,@edtech wonders- 3d printer in the classrooms ...,NaN,N/A,N/A,Neutral,English
53086,@electricimages_2015-01-28T22:00:46.000Z,3927752.0,Use-case,Other,Positive,English
...,...,...,...,...,...,...
2759098,@amigobulls_2015-08-18T16:07:38.000000Z,4291045.0,Business-relevant,Market Trends & Forecast,Positive,English
2782814,@Clement_MENGUE_2014-05-09T07:30:04.000Z,5230609.0,Business-relevant,Market Trends & Forecast,Positive,English
2811647,@3Dable_2022-02-02T16:06:32.000Z,6313660.0,Use-case,Medical / Dental,Positive,English
2820642,@tombielecki_2012-11-19 03:45:55.000Z,39155.0,N/A,N/A,Negative,English


## Check Unprocessed Rows

In [76]:
def check_missing_cleaned_rows(
    processed_df: pd.DataFrame,
    cleaned_df: pd.DataFrame,
    *,
    tweet_id_col: str = "tweet_id"
) -> pd.DataFrame:
    """
    Identify all rows in `cleaned_df` that are not present in `processed_df`
    based on the cleaned `tweet_id` (stripped, no trailing punctuation).

    Returns rows in cleaned_df where tweet_id is missing from processed_df.
    """
    processed_clean = processed_df.copy()
    cleaned_clean = cleaned_df.copy()

    # Clean tweet_id columns
    processed_clean[tweet_id_col] = clean_tweet_ids(processed_clean, tweet_id_col)
    cleaned_clean[tweet_id_col] = clean_tweet_ids(cleaned_clean, tweet_id_col)

    # Find tweet_ids in cleaned but not in processed
    missing_rows = cleaned_clean[
        ~cleaned_clean[tweet_id_col].isin(processed_clean[tweet_id_col])
    ].copy()

    return missing_rows


In [77]:
missing_from_processed = check_missing_cleaned_rows(
    processed_df=processed_data_df,
    cleaned_df=cleaned_df,
    tweet_id_col="tweet_id"
)
missing_from_processed

,Author ID,Total Engagement,Date,tweet_id,row_num,Normalized Text,langdetect_is_english
4,@rveenewman,213595,2019-01-09 13:25:07+00:00,@rveenewman_2019-01-09T13:25:07.000Z,2447524,a 3d printed light projected animation. proof ...,True
636,@BirnoOCE,2223,2021-08-29 17:43:48+00:00,@BirnoOCE_2021-08-29T17:43:48.000Z,2867008,heirloom giveaway win yourself a 3d printed he...,True
6662,@ThomasDangAB,280,2020-04-13 21:00:41+00:00,@ThomasDangAB_2020-04-13T21:00:41.000Z,420939,off to deliver ~500 3d printed ear savers to n...,True
9465,@nano,204,2021-10-10 17:00:11+00:00,@nano_2021-10-10T17:00:11.000Z,2888729,thanks to the collaboration between @newstoryc...,True
12756,@EOS3DPrinting,157,2017-10-18 14:58:19+00:00,@EOS3DPrinting_2017-10-18T14:58:19.000Z,1314705,mass production w/ #3dprinting is reality! we ...,True
...,...,...,...,...,...,...,...
2834438,@ProHacktivo,0,2021-07-09 17:51:32+00:00,@ProHacktivo_2021-07-09T17:51:32.000Z,2907693,cyber threat in 3d printing says pentagon,True
2834800,@DRSCornwall,0,2020-06-21 22:45:12+00:00,@DRSCornwall_2020-06-21T22:45:12.000Z,411560,customer using the new drs tuning st180 maf ad...,True
2834804,@TurboKitsdotcom,0,2020-04-13 16:10:05+00:00,@TurboKitsdotcom_2020-04-13T16:10:05.000Z,579709,"customer photo of our mercedes a, cla, gla 3d ...",True
2834879,@_Brandor,0,2016-08-05 01:04:46+00:00,@_Brandor_2016-08-05T01:04:46.000Z,1580187,customer centricity ... future pharmacy – 3d p...,True


In [78]:
len(missing_from_processed)

23109

In [88]:
matches = processed_data_df[
    processed_data_df["tweet_id"].astype(str).str.contains("mirrrroring_", na=False)
]

print(matches[["tweet_id"]])


                                      tweet_id
2575035  @mirrrroring_2013-08-11T21:42:57.000Z


In [89]:
null_or_empty_category_df = processed_data_df[
    processed_data_df["category"].isnull()
    | processed_data_df["category"].astype(str).str.strip().eq("")
    | processed_data_df["category"].str.upper().eq("N/A")
]
null_or_empty_category_df

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
54,@SaveethaSEC_2023-10-28T04:36:02.000Z,6393344.0,N/A,N/A,Neutral,English
61,@bufbvr_2017-05-12T03:33:06.000Z,1214766.0,N/A,N/A,Neutral,English
64,@DIWHYTHESHOW_2018-07-10T16:23:19.000Z,3167757.0,N/A,N/A,Neutral,English
65,@jariwalavi_2019-10-20T11:37:12.000Z,2428798.0,N/A,N/A,Neutral,English
66,@mystem3d_2019-10-23T04:21:06.000Z,2670662.0,N/A,N/A,Positive,English
...,...,...,...,...,...,...
2825961,@ProdigalSonDJR_2018-04-06T17:36:07.000Z,3271045.0,N/A,N/A,Negative,English
2825975,@kaerolize_2013-07-31T09:22:41.000Z,5818941.0,N/A,N/A,Positive,English
2825979,@1HDBIZ_2022-05-18T12:21:14.000Z,6544124.0,N/A,N/A,Positive,English
2825983,@3D_Print_NFT_2022-04-28T01:24:34.000Z,6589991.0,N/A,N/A,Neutral,English


In [79]:
processed_data_df

,tweet_id,row_num,category,subcategory,sentiment,tweet_language
0,@3HTi__2017-04-13T21:13:00.000Z,1938218.0,Business-relevant,Intellectual Property & Patents,Neutral,English
1,@benzuria_2017-06-12T16:33:14.000Z,1614375.0,Business-relevant,Intellectual Property & Patents,Neutral,English
2,@stephcurcio_2016-02-22T16:27:33.000Z,904891.0,Business-relevant,Intellectual Property & Patents,Positive,English
3,@patesalo_e_2017-06-10T10:26:13.000Z,697479.0,Business-relevant,Intellectual Property & Patents,Neutral,English
4,@eTeknix_2015-04-19T18:04:07.000Z,4456221.0,Use-case,Consumer Products / Electronics,Positive,English
...,...,...,...,...,...,...
2825997,@dgzitting_2016-07-20T22:36:24.000Z,2176450.0,Use-case,Architectural,Positive,English
2825998,@MartaKrivosheek_2015-10-31T10:45:52.000Z,3816338.0,Technological,Materials,Neutral,English
2825999,@anciana_abuela_2018-04-14T16:42:12.000Z,3209381.0,Technological,Materials,Positive,English
2826000,@crimsonns_2022-09-06T16:39:17.000Z,6467551.0,Business-relevant,Investment & Financing,Neutral,English


In [80]:
from datetime import datetime

# Generate a timestamped filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"CLEANED_POST-LLM_DATASET_v1__{timestamp}.csv"

# Write out the DataFrame to CSV
processed_data_df.to_csv(filename, index=False)

print(f"DataFrame successfully written to {filename}")


DataFrame successfully written to CLEANED_POST-LLM_DATASET_v1__20250603_111851.csv
